# Silver Data Transformation

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("delta")\
    .option("header", True)\
    .option("inferschema", True)\
    .load("abfss://bronze@netflixprojectdlvs.dfs.core.windows.net/netflix_titles")

In [0]:
display(df)

In [0]:
df = df.fillna({"duration_minutes" : 0, "duration_seasons" : 1})

In [0]:
df.display()

In [0]:
df = df.withColumn("duration_minutes", col('duration_minutes').cast(IntegerType()))\
        .withColumn("duration_seasons", col('duration_seasons').cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
df.display()

In [0]:
df = df.withColumn("shorttitle", split('title',':')[0])
df.display()

In [0]:
df = df.withColumn("rating", split('rating','-')[0])
df.display()

In [0]:
df = df.withColumn("type_flag", when(col('type') == 'Movie', 1)\
                       .when(col('type') == 'TV Show', 2)\
                       .otherwise(0))
df.display()

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import dense_rank, col, desc
from pyspark.sql.functions import count

In [0]:

 df = df.withColumn("duration_ranking", dense_rank().over(Window.orderBy(desc(col("duration_minutes")))))
 df.display()


In [0]:
df.createOrReplaceTempView("temp_view")

In [0]:
df = spark.sql("""
               
               select * from temp_view

               """)
df.display()

In [0]:
df.createOrReplaceGlobalTempView("global_view")

In [0]:
df = spark.sql("""
               
               select * from global_temp.global_view

               """)
df.display()

In [0]:
from pyspark.sql.functions import count
df_vis = df.groupBy("type").agg(count("*").alias("total_count"))
display(df_vis)

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .option("path", "abfss://silver@netflixprojectdlvs.dfs.core.windows.net/netflix_titles")\
        .save()